In [ ]:
!pip install vaderSentiment

In [32]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Install Model

In [33]:
analyzer = SentimentIntensityAnalyzer()

# Encode and Calculate Sentiment

In [38]:
score = analyzer.polarity_scores('Hello, My name is Bryan. I hate doing Sentiment Analysis')

In [39]:
sentiment = 1 + max(0, round(4 * (score['compound'] + 1) / 2))

In [40]:
int(sentiment)

2

# Collect Reviews from Yelp

In [9]:
reviews = []

In [10]:
for page in range(11):
  page_url = 'https://www.yelp.com/biz/vatos-urban-tacos-singapore?osq=Mexican&start={start}&sort_by=date_desc'.format(start=page*10)
  r = requests.get(page_url)
  soup = BeautifulSoup(r.text, 'html.parser')
  regex = re.compile('.*comment.*')
  results = soup.find_all('p', {'class':regex})
  reviews += [result.text.strip() for result in results]

In [11]:
len(reviews)

85

# Load Reviews into DF and Score

In [12]:
df = pd.DataFrame(np.array(reviews), columns=['Review'])

In [13]:
df.shape

(85, 1)

In [14]:
df.head(10)

,Review
0,"Hands down, the worst nachos I've ever had. Me..."
1,"The food was great, I highly recommend the jal..."
2,"A friend who used to live here recommended it,..."
3,Greta place to come chill but now can only tak...
4,Good food nice environment great selection alm...
5,Mexican food in Singapore? Yes absolutely Yes!...
6,"Well, Yelp never lets me down. Came to Vatos f..."
7,"Food has a fishy taste. I'm from California, ..."
8,This place is just mediocre. The food isn't as...
9,Who knew I would be drinking a beer from North...


# Apply VADER model on reviews for sentiment analysis

In [28]:
analyzer = SentimentIntensityAnalyzer()

In [29]:
def sentiment_score(review):
    # Compute the sentiment scores using VADER
    scores = analyzer.polarity_scores(review)

    # Map the compound score to a 5-point scale (1 to 5)
    sentiment = 1 + max(0, round(4 * (scores['compound'] + 1) / 2))
    return int(sentiment)

In [30]:
df['Sentiment'] = df['Review'].apply(lambda x: sentiment_score(x))

In [31]:
df.head(10)

,Review,Sentiment
0,"Hands down, the worst nachos I've ever had. Me...",1
1,"The food was great, I highly recommend the jal...",5
2,"A friend who used to live here recommended it,...",5
3,Greta place to come chill but now can only tak...,4
4,Good food nice environment great selection alm...,5
5,Mexican food in Singapore? Yes absolutely Yes!...,5
6,"Well, Yelp never lets me down. Came to Vatos f...",5
7,"Food has a fishy taste. I'm from California, ...",3
8,This place is just mediocre. The food isn't as...,2
9,Who knew I would be drinking a beer from North...,4
